In [1]:
import matplotlib
matplotlib.use('Agg')

import numpy as np
import keras
import tensorflow as tf
import itertools    

from keras.layers.advanced_activations import LeakyReLU, PReLU, ELU
from keras.layers import Conv2D, MaxPooling2D, Flatten, Input, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l1,l2

from keras.models import Model
from keras.callbacks import TensorBoard
from keras import backend as K
from keras.engine.topology import Layer

from sklearn.metrics import classification_report, confusion_matrix

import matplotlib.pyplot as plt
import load_data

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.
sys.version_info(major=2, minor=7, micro=6, releaselevel='final', serial=0)


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
keras.backend.set_session(sess)

In [3]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix',cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [4]:
def plot_classwise(y_test, y_pred, name):
    y_test_aho = np.argmax(y_test, axis=1)
    y_pred_aho = np.argmax(y_pred, axis=1)
    print(classification_report(y_test_aho, y_pred_aho))
    plt.figure()
    cnf_matrix = confusion_matrix(y_test_aho, y_pred_aho)
    plot_confusion_matrix(cnf_matrix, range(10),title='Confusion matrix')
    plt.savefig(name)

In [5]:
class ChannelIgnore(Layer):

    def __init__(self, drop_ratio=2, **kwargs):
        super(ChannelIgnore, self).__init__(**kwargs)
        assert(drop_ratio>0)
        self.drop_ratio = drop_ratio

    def build(self, input_shape):
        super(ChannelIgnore, self).build(input_shape)

    def call(self, x):
        y = x[:]
        if K.image_dim_ordering() == 'tf':
            channels = np.random.choice(x.shape[3], x.shape[3]/self.drop_ratio, replace=False)
            for i in range(y.shape[0]):
                y[i,:,:,channels] = 0.0
        else:
            channels = np.random.choice(x.shape[1], x.shape[3]/self.drop_ratio, replace=False)
            for i in range(y.shape[0]):
                y[:,channels,:,:] = 0.0
        return K.in_train_phase(y,x)

    def compute_output_shape(self, input_shape):
        return input_shape


In [6]:
def build_model(regularizer=None):
    input_layer = Input(shape=(96,96,3))
    
    model = Conv2D(64, (3, 3), activation='tanh',kernel_regularizer=regularizer)(input_layer)
    model = Conv2D(64, (3, 3), activation='tanh',kernel_regularizer=regularizer)(model)
    model = MaxPooling2D()(model)
    model = Dropout(0.5)(model)
    model = BatchNormalization()(model)
    
    model = Conv2D(128, (3, 3), activation='tanh',kernel_regularizer=regularizer)(model)
    model = Conv2D(128, (3, 3), activation='tanh',kernel_regularizer=regularizer)(model)
    model = MaxPooling2D()(model)
    model = Dropout(0.5)(model)
    model = BatchNormalization()(model)
    
    model = Conv2D(256, (3, 3), activation='tanh',kernel_regularizer=regularizer)(model)
    model = Conv2D(256, (3, 3), activation='tanh',kernel_regularizer=regularizer)(model)
    model = MaxPooling2D()(model)
    model = Dropout(0.5)(model)
    model = BatchNormalization()(model)
    
    model = Flatten()(model)
    model = Dense(10,kernel_regularizer=regularizer, activation='softmax')(model)
    
    network = Model(input_layer, model)
    opt = keras.optimizers.Adadelta()
    network.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])
    return network

In [7]:
def build_novel_model():
    input_layer = Input(shape=(96,96,3))
    
    model = ChannelIgnore()(input_layer)
    model = Conv2D(64, (3, 3), activation='tanh',kernel_regularizer=regularizer)(model)
    model = ChannelIgnore()(input_layer)
    model = Conv2D(64, (3, 3), activation='tanh',kernel_regularizer=regularizer)(model)
    model = ChannelIgnore()(input_layer)
    model = MaxPooling2D()(model)
    model = BatchNormalization()(model)
    
    model = ChannelIgnore()(input_layer)
    model = Conv2D(128, (3, 3), activation='tanh',kernel_regularizer=regularizer)(model)
    model = ChannelIgnore()(input_layer)
    model = Conv2D(128, (3, 3), activation='tanh',kernel_regularizer=regularizer)(model)
    model = ChannelIgnore()(input_layer)
    model = MaxPooling2D()(model)
    model = BatchNormalization()(model)
    
    model = ChannelIgnore()(input_layer)
    model = Conv2D(256, (3, 3), activation='tanh',kernel_regularizer=regularizer)(model)
    model = ChannelIgnore()(input_layer)
    model = Conv2D(256, (3, 3), activation='tanh',kernel_regularizer=regularizer)(model)
    model = ChannelIgnore()(input_layer)
    model = MaxPooling2D()(model)
    model = BatchNormalization()(model)
    
    model = ChannelIgnore()(input_layer)
    model = Flatten()(model)
    model = Dense(10,kernel_regularizer=regularizer, activation='softmax')(model)
    
    network = Model(input_layer, model)
    opt = keras.optimizers.Adadelta()
    network.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])
    return network

In [8]:
def partA1(x_train, y_train, x_test, y_test):
    model = build_model()
    model.fit(x_train, y_train, batch_size=2,epochs=100,validation_split=0.2,callbacks=[TensorBoard('./partA1')])
    y_pred = model.predict(x_test)
    plot_classwise(y_test, y_pred, 'partA1.png')

In [9]:
def partA2(x_train, y_train, x_test, y_test):
    model = build_model()
    model.fit(x_train, y_train, batch_size=2,epochs=100,validation_split=0.2,callbacks=[TensorBoard('./partA2',histogram_freq=100)])
    y_pred = model.predict(x_test)
    plot_classwise(y_test, y_pred, 'partA2.png')
    return y_pred

In [10]:
def partA3(x_train, y_train, x_test, y_test):
    #L1 regularization
#     model = build_model('l1')
#     model.fit(x_train, y_train, batch_size=2,epochs=100,validation_split=0.2,callbacks=[TensorBoard('./partA3a',histogram_freq=100)])
#     y_pred = model.predict(x_test)
#     plot_classwise(y_test, y_pred, 'partA3a.png')
#     #L2 regularization
#     model = build_model('l2')
#     model.fit(x_train, y_train, batch_size=2,epochs=100,validation_split=0.2,callbacks=[TensorBoard('./partA3b',histogram_freq=100)])
#     y_pred = model.predict(x_test)
#     plot_classwise(y_test, y_pred, 'partA3b.png')
    #Custom regularization
    model = build_novel_model()
    model.fit(x_train, y_train, batch_size=2,epochs=100,validation_split=0.2,callbacks=[TensorBoard('./partA3c',histogram_freq=100)])
    y_pred = model.predict(x_test)
    plot_classwise(y_test, y_pred, 'partA3c.png')
    return y_pred

In [11]:
def compare_predictions(y_pred_noreg, y_pred_reg, y_test):
    reg_correct = (np.argmax(y_pred_reg, axis=1) == np.argmax(y_test, axis=1))
    reg_incorrect = (np.argmax(y_pred_reg, axis=1) != np.argmax(y_test, axis=1))
    nonreg_incorrect = (np.argmax(y_pred_noreg, axis=1) != np.argmax(y_test, axis=1))
    A = np.logical_and(reg_correct, nonreg_incorrect)
    B = np.logical_and(reg_incorrect, nonreg_incorrect)
    return np.nonzero(A)[0], np.nonzero(B)[0]

In [12]:
def save_images(prefix, images):
    from PIL import Image
    for i, image in enumerate(images):
        result = Image.fromarray((image * 255).astype('uint8'))
        result.save(prefix + str(i+1) + '.bmp')

In [ ]:
(X_train, Y_train), (X_test, Y_test), cross_val_indices = load_data.load()
X_train_sub = X_train[cross_val_indices[0]]
Y_train_sub = Y_train[cross_val_indices[0]]

In [ ]:
partA1(X_train, Y_train, X_test, Y_test)

no_regularize = partA2(X_train_sub, Y_train_sub, X_test, Y_test)

regularize = partA3(X_train_sub, Y_train_sub, X_test, Y_test)

parta, partb = compare_predictions(no_regularize, regularize, Y_test)

inspect_A = X_test[parta]
inspect_B = X_test[partb]

Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 29s - loss: 3.0243 - acc: 0.1813 - val_loss: 3.6987 - val_acc: 0.1750
Epoch 2/100
800/800 [==============================] - 22s - loss: 2.9000 - acc: 0.2087 - val_loss: 3.8397 - val_acc: 0.1050
Epoch 3/100
800/800 [==============================] - 20s - loss: 2.7390 - acc: 0.2675 - val_loss: 2.8668 - val_acc: 0.2200
Epoch 4/100
800/800 [==============================] - 21s - loss: 2.3780 - acc: 0.3400 - val_loss: 3.2628 - val_acc: 0.2100
Epoch 5/100
800/800 [==============================] - 22s - loss: 2.2198 - acc: 0.3613 - val_loss: 3.0159 - val_acc: 0.2200
Epoch 6/100
800/800 [==============================] - 22s - loss: 2.1093 - acc: 0.3925 - val_loss: 2.9040 - val_acc: 0.2650
Epoch 7/100
800/800 [==============================] - 20s - loss: 1.8068 - acc: 0.4588 - val_loss: 2.6660 - val_acc: 0.2800
Epoch 8/100
176/800 [=====>........................] - ETA: 16s - loss: 1.4497 

In [ ]:
save_images("iparta/",inspect_A)
save_images("ipartb/",inspect_B)